In [ ]:
import pymysql
import pandas as pd
import numpy as np
from GraphConstruct import GraphConstruct
from matplotlib import pyplot as plt
import networkx as nx
# 网络里如果发票信息不全，有空字符串存在，读入的时候就是None，如果某个节点是None的话，就会出现很多问题

In [ ]:
connection = pymysql.connect(
    host = '127.0.0.1',
    user = '***', 
    password = '******',
    database = '****',
    port = 3306,
    charset = 'utf8mb4', 
    cursorclass = pymysql.cursors.Cursor)

In [ ]:
try:
    with connection.cursor() as cursor:
        # Read a single record
        sql = "SELECT XFNSRDZDAH, GFNSRDZDAH from tax.VAT_invoice"
        cursor.execute(sql)
        result = cursor.fetchall()
        # 用 'XUJIAFAPIAO' 代替 None
        edges = []
        n = 0
        for edge in result:
            edge = list(edge)
            if edge[0] == None:
                edge[0] = 'XUJIAFAPIAO'
                n += 1
            if edge[1] == None:
                edge[1] = 'XUJIAFAPIAO'
                n += 1
            edges.append(edge)
        print('None exist {} times'.format(n))
        result = pd.DataFrame(edges)
        result.drop_duplicates(inplace=True) # remove duplicate
        result = result.values

finally:
    print('get 销方纳税人电子档案号和购方纳税人电子档案号！')
    #connection.close()

In [ ]:
result

In [ ]:
try:
    with connection.cursor() as cursor:
        # Read a single record
        sql = "SELECT NSRDZDAH FROM tax.XYQY"
        cursor.execute(sql)
        xukai = cursor.fetchall()
        #print(result[0][0])
finally:
    print('获取虚开名单')
    #connection.close()

In [ ]:
new_xukai = []
for item in xukai:
    new_xukai.append(item[0])
new_xukai.append('XUJIAFAPIAO')

In [ ]:
container = GraphConstruct()

In [ ]:
G = container.construct_network(result)

In [ ]:
# 为网络的节点添加属性
for node in G.nodes():
    G.add_node(node, xukai=False)
    
for node in new_xukai:
    G.add_node(node, xukai=True)

In [ ]:
# in degree 其实没有入度和出度，因为是无向图
in_degrees = list(G.degree())
in_degree = [value[1] for value in in_degrees]
# max degree of the network
in_degree_num = {}
for i in in_degree:
    if i in in_degree_num:
        in_degree_num[i] += 1
    else:
        in_degree_num[i] = 1    
x = sorted(in_degree_num)
y = [in_degree_num[k] for k in x]
y = np.array(y) / G.number_of_nodes()
save = pd.DataFrame({'x':x, 'y':y})
save.to_pickle('./Figure/hb.pickle')
plt.xlabel('Degree')
plt.ylabel('Count')
plt.loglog(x, y)
plt.savefig('./Figure/HeBei.pdf')
# 度分布呈现幂率分布特性，典型的无标度网络
plt.show()
sum_degree = 0
for i in range(len(x)):
    sum_degree = sum_degree + x[i] * y[i]
average_degree = sum_degree / len(G.nodes())
print('average degree', average_degree)

In [ ]:
# 集聚系数 平均集聚系数
#随机网络的平均聚集系数Crand为：0.0003463，C/Crand=78.689，所以C>>Crand，该社交网络的平均聚集系数这一特征也符合小世界网络的特征。
# subgraphs_nodes = nx.connected_components(G)
# subgraphs = []
# for c in sorted(subgraphs_nodes, key=len,reverse=True):
#     subgraphs.append(G.subgraph(c))
# print(subgraphs[0].edges())
# clusters = nx.clustering(subgraphs[0], nodes=None)
# print(clusters)
#print(type(list(G.nodes())[0]))
avg_clusters = nx.average_clustering(G)
avg_clusters

In [ ]:
# cliques   团，小圈子

cliques = nx.graph_number_of_cliques(G)

cliques

In [ ]:

# density

density = nx.density(G)

density

# 密度非常低表示边的连接比较稀疏


In [ ]:
subgraphs_nodes = nx.connected_components(G)
subgraphs = []
for c in sorted(subgraphs_nodes, key=len,reverse=True):
    print(len(c))
    subgraphs.append(G.subgraph(c))

print('number of subgraphs {}'.format(len(subgraphs)))

In [ ]:

# 直径

# 直径表示交易的长度

# diameter = nx.diameter(subgraphs[0])

# diameter

# 中心度

# 1， 点中心度， 2， 紧密中心度， 3， 介数中心度， 4， 特征向量中心度

In [ ]:
import community
def get_subgraphs(G):
    subgraphs_nodes = nx.connected_components(G)
    subgraphs = []
    for c in sorted(subgraphs_nodes, key=len,reverse=True):
        subgraphs.append(G.subgraph(c))
    new_subgraphs = []
    new_subgraphs = new_subgraphs + subgraphs[1:]
    partitions = community.best_partition(subgraphs[0])
    for com in set(partitions.values()):
        list_nodes = [nodes for nodes in partitions.keys() if partitions[nodes] == com]
        new_subgraphs.append(nx.Graph(subgraphs[0].subgraph(list_nodes).edges()))
    new_subgraphs = sorted(new_subgraphs, key=len, reverse=True)
    print(len(new_subgraphs))
    for i, graph in enumerate(new_subgraphs) :
        if len(graph.nodes) > 1000:
            print('{} graph information'.format(i))
            print('nodes number {}'.format(len(graph.nodes())))
            print('edges number {}\n\n'.format(len(graph.edges())))
            nx.write_gexf(graph, './Figure/ShangHai/' +str(i) + '.gexf')
    return new_subgraphs

In [ ]:
a = get_subgraphs(G)

In [ ]:
def get_uniform_node_number_subgraphs(G):
    import community
    subgraphs_nodes = nx.connected_components(G)
    subgraphs = []
    for c in sorted(subgraphs_nodes, key=len,reverse=True):
        ap_graph = nx.Graph()
        ap_graph.add_nodes_from(c)
        ap_graph.add_edges_from(G.subgraph(c).edges())
        subgraphs.append(ap_graph)
    new_subgraphs = []
    new_subgraphs = new_subgraphs + subgraphs[1:]
    partitions = community.best_partition(subgraphs[0])
    for com in set(partitions.values()):
        list_nodes = [nodes for nodes in partitions.keys() if partitions[nodes] == com]
        ap_graph = nx.Graph()
        ap_graph.add_nodes_from(list_nodes)
        ap_graph.add_edges_from(subgraphs[0].subgraph(list_nodes).edges())
        new_subgraphs.append(ap_graph)
    new_subgraphs = sorted(new_subgraphs, key=len)


    print('total subgraphs ', len(new_subgraphs))
    result_subgraphs = []
    insert_subgraphs = []
    total_nodes = 0
    index = 0
    for g in new_subgraphs:
        if index == 0:
            insert_subgraphs.append(g)
            total_nodes += g.number_of_nodes()
            index += 1
            continue
        if total_nodes > 1000:
            result_subgraphs.append(conn_graphs(insert_subgraphs))
            total_nodes = 0
            insert_subgraphs = []
            index += 1
            insert_subgraphs.append(g)
        else:
            insert_subgraphs.append(g)
            total_nodes += g.number_of_nodes()
            index += 1
    if total_nodes > 0:
        result_subgraphs.append(conn_graphs(insert_subgraphs))
    result_subgraphs = sorted(result_subgraphs, key=len)
    total_num = 0
    for gr in result_subgraphs:
        total_num += gr.number_of_nodes()
    print('result nodes, ', total_num)
    print('final get subgraphs ', len(result_subgraphs))
    return result_subgraphs

In [ ]:
b = get_uniform_node_number_subgraphs(G)